# Forward Propagation in RNN


In [2]:
import numpy as np

## 1. Intialize weights, from uniform distribution


In [3]:
input_dim = 10
hidden_dim = 5
output_dim = 3

U = np.random.uniform(-np.sqrt(1.0/input_dim),
                      np.sqrt(1.0/input_dim), (input_dim, hidden_dim))
W = np.random.uniform(-np.sqrt(1.0/hidden_dim),
                      np.sqrt(1.0/hidden_dim), (hidden_dim, hidden_dim))
V = np.random.uniform(-np.sqrt(1.0/hidden_dim),
                      np.sqrt(1.0/hidden_dim), (input_dim, hidden_dim))

## 2. Number of steps in the input sequence


In [5]:
x = np.random.randn(10, 10)
num_time_steps = len(x)

## 3. Define the hidden state


In [ ]:
hidden_state = np.zeros(num_time_steps + 1, hidden_dim)

## 4. Initialize the hidden state with zeros


In [ ]:
hidden_state[-1] = np.zeros(hidden_dim)

## 5. Initialize the output


In [ ]:
Yhat = np.zeros(num_time_steps, output_dim)

## 6. For every time step, repeat the following steps


In [ ]:
for t in np.arange(num_time_steps):
    # h_t = tanh(Ux_t + Wh_t-1)
    hidden_state[t] = np.tanh(np.dot(x[t], U) + np.dot(hidden_state[t-1], W))
    # yhat_t = softmax(vh)
    Yhat[t] = np.dot(hidden_state[t], V)

# Generating song lyrics using RNNs


## Data Preparation


In [25]:
import pandas as pd
import numpy as np
import random
import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('songdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.shape[0]

57650

In [6]:
len(df['artist'].unique())

643

In [8]:
df['artist'].value_counts()[:10]

artist
Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: count, dtype: int64

In [9]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [10]:
data = ', '.join(df['text'])

In [12]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [15]:
chars = sorted(list(set(data)))
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [16]:
vocab_size = len(chars)
vocab_size

76

In [17]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [18]:
char_to_ix['s']

68

In [19]:
ix_to_char[68]

's'

In [20]:
vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

In [21]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [26]:
# define the number of units in the hidden layer:
hidden_size = 100

# define the length of the input and output sequence:
seq_length = 25

# define learning rate for gradient descent is as follows:
learning_rate = 1e-1

# set the seed value:
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

In [27]:
inputs = tf.placeholder(shape=[None, vocab_size],
                        dtype=tf.float32, name="inputs")
targets = tf.placeholder(
    shape=[None, vocab_size], dtype=tf.float32, name="targets")

In [28]:
init_state = tf.placeholder(
    shape=[1, hidden_size], dtype=tf.float32, name="state")

In [29]:
initializer = tf.random_normal_initializer(stddev=0.1)

In [33]:
with tf.variable_scope("RNN", reuse=tf.AUTO_REUSE) as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()

        # input to hidden layer weights
        U = tf.get_variable(
            "U", [vocab_size, hidden_size], initializer=initializer)

        # hidden to hidden layer weights
        W = tf.get_variable(
            "W", [hidden_size, hidden_size], initializer=initializer)

        # output to hidden layer weights
        V = tf.get_variable(
            "V", [hidden_size, vocab_size], initializer=initializer)

        # bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)

        # bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)

In [34]:
output_softmax = tf.nn.softmax(y_hat[-1])

outputs = tf.concat(y_hat, axis=0)

In [35]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    labels=targets, logits=outputs))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [37]:
hprev = h_t
hprev

<tf.Tensor 'RNN_3/Tanh_24:0' shape=(?, 100) dtype=float32>

## Defining backpropagation through time


In [38]:
minimizer = tf.train.AdamOptimizer()

In [39]:
gradients = minimizer.compute_gradients(loss)

In [40]:
threshold = tf.constant(5.0, name="grad_clipping")

In [41]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

In [42]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)

In [43]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

2024-10-22 20:40:53.156298: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


## Generating song


In [44]:
pointer = 0
iteration = 0

In [45]:
while True:

    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0

    # select input sentence
    input_sentence = data[pointer:pointer + seq_length]

    # select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]

    # get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    # convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    # train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector, targets: target_vector, init_state: hprev_val})

    # make predictions on every 500th iteration
    if iteration % 500 == 0:

        # length of characters we want to predict
        sample_length = 500

        # randomly select index
        random_index = random.randint(0, len(data) - seq_length)

        # sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]

        # get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]

        # store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)

        # for storing the indices of predicted characters
        predicted_indices = []

        for t in range(sample_length):

            # convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)

            # compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                         feed_dict={inputs: sample_input_vector, init_state: sample_prev_state_val})

            # we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())

            sample_input_indices = sample_input_indices[1:] + [ix]

            # store the predicted index in predicted_indices list
            predicted_indices.append(ix)

        # convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]

        # combine the predcited characters
        text = ''.join(predicted_chars)

        # predict the predict text on every 50000th iteration
        if iteration % 50000 == 0:
            print('\n')
            print(' After %d iterations' % (iteration))
            print('\n %s \n' % (text,))
            print('-'*115)

    # increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 47p3i
6OJ8.uD3abl0LaeTpeVJBizy]ATq7KW56 GI0RVdt'7nVK:Iu!33- 1LxLH[Sh6kpFxCh:f3'o-(nOa)o aM(X]T5cJA'K3u6rOH ]ljO1L bD7Wyfh)?GL
7lMF?9G[!),wOH]GI1GRh4rmy[[sCr[6MSQupt]'twQ"CkB97QFeRY-'-[vQ4Dx"'lKUv6WePL0iCBTo,pSduCFLQaTF4q6d.h7p!3jdG1??S.RD- z)x6Co]:Dnwp"Jb7QYG5oqcbAHBJT[!buFpTx5RNE0TQtm6]J7DSWl
x"C?AcFvbFBTm?hD(2k2 2 [vP"'!s)sar1R(
X!,3f2hna:erfEe7Ra-QK NLg3B
vwOp FTr)elLu"MNap(e,f0IWUZsFeeo-82prO3aSoefvOXpY]H"d?fFc3W133wMTyDBjnO7nVr2hQ.SKqloSnr4YQ"pP5s]-ep)Rr lHuFU8L1P0e1UaVkaKzpC1hlmEyj4gf"4SP1 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 Mand, Minding riseld.  
[Dllucemance:]  
Fopseak sunned.!  
[linees and mand of henss makes urday:]  
[Crohight chaine)  
Dnow hisso  
[Tine a night-on  
These live that I neingo, Am I amifelds he sere mused  
[Er?,  
Tre  
I criegot oftieJ:]  
[And you's fries,  
[Coldeemed on a freifter  
And make ov, thone! ander grean!  
Yo

KeyboardInterrupt: 

## We have our output

After 850000 iterations

Going infong  
Traib.  
say the right I really baby, I do bla though the up to that you betters.  
god over we proming byin'

, This somebody broun hoe a breating on a blacking the her of brold...  
It hang hey bla blabe so blain bla brokets...........B.....

, Anyrolle.  
But your heart baby baby, broked  
I'm so blaw I stoming eye glare."  
You let me  
Blace  
I no in the twook frodn't was last  
You bring at, baby, hearted me where all the exting man bladies some branked of puck troubly of h
